[View in Colaboratory](https://colab.research.google.com/github/sadimanna/CNN_Tf/blob/master/vgg13.ipynb)

Import libraries

In [0]:
import numpy as np
import tensorflow as tf
import time, random
import matplotlib.pyplot as plt

from google.colab import files

import warnings

Suppress Warnings

In [0]:
warnings.simplefilter("ignore", DeprecationWarning)
warnings.simplefilter("ignore", FutureWarning)

**Define Support Functions**

***Function for Weights***

In [0]:
def get_weights(name,shape):
	return tf.get_variable(name=name,shape=shape,initializer = tf.contrib.layers.xavier_initializer(uniform=False),regularizer = tf.contrib.layers.l2_regularizer(tf.constant(0.0005, dtype=tf.float32)))


***Function for Biases***

In [0]:
def get_bias(name,shape):
	return tf.get_variable(name=name,shape=shape,initializer = tf.zeros_initializer())

***Convolutional Layer Function***

In [0]:
def conv2d(inp,name,kshape,s):
	with tf.variable_scope(name) as scope:
		kernel = get_weights('weights',shape=kshape)
		conv = tf.nn.conv2d(inp,kernel,[1,s,s,1],'SAME')
		bias = get_bias('biases',shape=kshape[3])
		preact = tf.nn.bias_add(conv,bias)
		convlayer = tf.nn.relu(preact,name=scope.name)
	return convlayer

***Max Pool layer Function***

In [0]:
def maxpool(inp,name,k,s):
	return tf.nn.max_pool(inp,ksize=[1,k,k,1],strides=[1,s,s,1],padding='SAME',name=name)

***Loss Function***

In [0]:
def loss(logits,labels):
	labels = tf.reshape(tf.cast(labels,tf.int64),[-1])
	#print labels.get_shape().as_list(),logits.get_shape().as_list()
	cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(labels=labels,logits=logits,name='cross_entropy_per_example')
	cross_entropy_mean = tf.reduce_mean(cross_entropy,name='cross_entropy')
	total_loss = tf.add(tf.reduce_sum(tf.get_collection(tf.GraphKeys.REGULARIZATION_LOSSES)),cross_entropy_mean,name='total_loss')
	return total_loss

***Accuracy Function***

In [0]:
def accuracy(logits,true_labels):
	pred_labels = tf.argmax(logits,1)
	true_labels = tf.cast(true_labels,tf.int64)
	#print pred_labels.get_shape().as_list(),true_labels
	correct_pred = tf.cast(tf.equal(pred_labels, true_labels), tf.float32)
	accuracy = tf.reduce_mean(tf.cast(correct_pred,tf.float32))
	return accuracy

***Augmentation Supplementary Functions***

In [0]:
def get_new_size():
	new_size = 96 + random.choice([24,16,0])
	return [new_size,new_size]
	
def get_random_augmentation_combinations(length):
	out = [False,True]
	return [random.choice(out) for i in xrange(length)]

***Function for getting Training Images***

In [0]:
def get_all_images(img_file):
	images = np.fromfile(img_file,dtype=np.uint8).astype(np.float32)
	images = np.reshape(images,(-1,3,96,96))
	images = np.transpose(images,(0,3,2,1))
	print 'Normalizing Inputs...'
	rmean = np.mean(images[:,:,:,0])
	gmean = np.mean(images[:,:,:,1])
	bmean = np.mean(images[:,:,:,2])
	rstd = np.std(images[:,:,:,0])
	gstd = np.std(images[:,:,:,1])
	bstd = np.std(images[:,:,:,2])	
	images[:,:,:,0] = (images[:,:,:,0] - rmean)#/rstd
	images[:,:,:,1] = (images[:,:,:,1] - gmean)#/gstd
	images[:,:,:,2] = (images[:,:,:,2] - bmean)#/bstd
	print 'R_mean:',rmean,'G_mean:',gmean,'B_mean:',bmean
	print 'R_stddev:',rstd,'G_stddev:',gstd,'B_stddev:',bstd
	return images,rmean,gmean,bmean

***Function for getting Labels***

In [0]:
def get_all_labels(label_file):
	labels = np.fromfile(label_file,dtype=np.uint8)
	#print labels.shape
	return labels

***Function for getting Test Images***

In [0]:
def get_test_images(img_file,rmean,gmean,bmean):
	images = np.fromfile(img_file,dtype=np.uint8).astype(np.float32)
	images = np.reshape(images,(-1,3,96,96))
	images = np.transpose(images,(0,3,2,1))
	print 'Normalizing Validation Images...'
	images[:,:,:,0] = (images[:,:,:,0] - rmean)#/rstd
	images[:,:,:,1] = (images[:,:,:,1] - gmean)#/gstd
	images[:,:,:,2] = (images[:,:,:,2] - bmean)#/bstd
	return images

**Uploading Local Data**

In [0]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))

**Getting the Data**

In [0]:
train_data_path='train_X.bin'
train_label_path='train_y.bin'

train_img_file=open(train_data_path,'rb')
train_label_file=open(train_label_path,'rb')

train_x,rmean,gmean,bmean = get_all_images(train_img_file)
train_y = get_all_labels(train_label_file)
train_y = train_y-1

#Getting Validation Dataset
test_img_path = 'test_X.bin'
test_label_path = 'test_y.bin'

test_img_file = open(test_img_path,'rb')
test_label_file = open(test_label_path,'rb')

test_x = get_test_images(test_img_file,rmean,gmean,bmean)
test_y = get_all_labels(test_label_file)
print'Getting Validation set from Test set...'
val_x = test_x[300:500]
val_y = test_y[300:500]
val_y = val_y-1 #Label values converted from [1,10] to [0,10)

test_x = test_x[0:300]
test_y = test_y[0:300]
test_y = test_y-1

index = np.arange(train_x.shape[0])

**Variables and Placeholders**

In [0]:
init_count = 0
num_epochs = 100
batch_size = 50
numiter = 100
ne = 0
valacc = []
#Create Placeholders
feed_images = tf.placeholder(tf.float32,shape=(None,96,96,3))
feed_labels = tf.placeholder(tf.float32,shape=(None,))

lr = tf.placeholder(tf.float32,shape=())
keep_prob = tf.placeholder(tf.float32,shape=())

aug_img = tf.placeholder(tf.float32,shape=(96,96,3))

***Architecture***

In [0]:
#VGG13 Net Model
conv1 = conv2d(feed_images,'conv1',[3,3,3,64],1)
conv2 = conv2d(conv1,'conv2',[3,3,64,64],1)
pool1 = maxpool(conv2,'pool1',2,2)
#size = [N,48,48,64]
conv3 = conv2d(pool1,'conv3',[3,3,64,128],1)
conv4 = conv2d(conv3,'conv4',[3,3,128,128],1)
pool2 = maxpool(conv4,'pool2',2,2)
#size = [N,24,24,128]
conv5 = conv2d(pool2,'conv5',[3,3,128,256],1)
conv6 = conv2d(conv5,'conv6',[3,3,256,256],1)
conv7 = conv2d(conv6,'conv7',[3,3,256,256],1)
pool3 = maxpool(conv7,'pool3',2,2)
#size = [N,12,12,256]
conv8 = conv2d(pool3,'conv8',[3,3,256,512],1)
conv9 = conv2d(conv8,'conv9',[3,3,512,512],1)
conv10 = conv2d(conv9,'conv10',[3,3,512,512],1)
pool4 = maxpool(conv10,'pool4',2,2)
#size = [N,6,6,512]
conv11 = conv2d(pool4,'conv11',[3,3,512,512],1)
conv12 = conv2d(conv11,'conv12',[3,3,512,512],1)
conv13 = conv2d(conv12,'conv13',[3,3,512,512],1)
pool5 = maxpool(conv13,'pool5',2,2)
#size = [N,3,3,512]
pool5shape = pool5.get_shape().as_list()
N = pool5shape[1]*pool5shape[2]*pool5shape[3]
flattened_pool5 = tf.contrib.layers.flatten(pool5)
fc1 = tf.contrib.layers.fully_connected(flattened_pool5,4096,weights_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005, dtype=tf.float32)))
dropout1 = tf.nn.dropout(fc1,keep_prob)
fc2 = tf.contrib.layers.fully_connected(dropout1,4096,weights_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005, dtype=tf.float32)))
dropout2 = tf.nn.dropout(fc2,keep_prob)
logits = tf.contrib.layers.fully_connected(dropout2,10,activation_fn=None,weights_regularizer=tf.contrib.layers.l2_regularizer(tf.constant(0.0005, dtype=tf.float32)))